In [ ]:
# %pip install -q tfds-nightly keras tensorflow
# %pip install -q seaborn
# %pip install -q tf2onnx
# %pip install -q onnxmltools
# %pip install --upgrade pip

In [1]:
import pandas as pd
import numpy as np
import matplotlib.dates as date
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
import keras
import matplotlib.dates as date
import tensorflow as tf
from datetime import datetime
import seaborn as sns
import onnxmltools

<h1> Read, split, preprocces </h1>

In [3]:
dataPATH = "lab_window3_RssiPerCrownPerTime.csv"
# dataPATH = "lab_window5_RssiPerCrownPerTime.csv"
# dataPATH = "lab_window10_RssiPerCrownPerTime.csv"
# dataPATH = "lab_window20_RssiPerCrownPerTime.csv"
# dataPATH = "lab_window30_RssiPerCrownPerTime.csv"
# dataPATH = "lab_window40_RssiPerCrownPerTime.csv"
# dataPATH = "lab_window50_RssiPerCrownPerTime.csv"
# dataPATH = "lab_window60_RssiPerCrownPerTime.csv"

df = pd.read_csv(dataPATH, index_col=0)
print(df)

n_crownstones = 3
title = "3"

# manual enter feature columns
feature_names = [f"{i}.0" for i in range(n_crownstones)] + [f"mean{i}" for i in range(n_crownstones)] + [f"std{i}" for i in range(n_crownstones)]
# feature_names.append("")

features = df[feature_names].copy()

label = df[['posX', 'posY']].copy()  # labels x and y'

X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=0.2, random_state=1, stratify=label)

# optional
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
#

columnames = list(features.columns)+list(label.columns)
# sns.pairplot(df[columnames], diag_kind='kde')

                                   time        0.0        1.0        2.0  \
0      2023-07-05 17:27:59.357000+02:00 -72.609756 -74.909091 -74.450000   
1      2023-07-05 17:27:59.457000+02:00 -72.414634 -74.909091 -74.450000   
2      2023-07-05 17:27:59.557000+02:00 -72.219512 -74.909091 -74.450000   
3      2023-07-05 17:27:59.657000+02:00 -72.024390 -74.909091 -74.450000   
4      2023-07-05 17:27:59.757000+02:00 -71.829268 -74.909091 -74.450000   
...                                 ...        ...        ...        ...   
14990  2023-07-05 17:52:58.357000+02:00 -71.000000 -68.363636 -55.761905   
14991  2023-07-05 17:52:58.457000+02:00 -71.000000 -68.090909 -55.809524   
14992  2023-07-05 17:52:58.557000+02:00 -71.000000 -67.818182 -55.857143   
14993  2023-07-05 17:52:58.657000+02:00 -71.000000 -67.545455 -55.904762   
14994  2023-07-05 17:52:58.757000+02:00 -71.000000 -67.272727 -55.952381   

           mean0      std0      mean1      std1      mean2      std2  posX  \
0     -72

<h1> Layered Models Build </h1>

In [ ]:
model = keras.Sequential([
    keras.layers.Dense(64, input_dim=9, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(2)  # Output layer with 3 units for (x, y) later add , z)
])

model.compile(loss='mean_squared_error', optimizer='adam')

<h1> Training </h1>

In [ ]:
# Train the model
history = model.fit(X_train, y_train, validation_split = 0.2, epochs=50, verbose = 0)

<h1> Evaluating model and Plotting </h1>

In [ ]:
def plot_loss(history):
  plt.plot(history.history['loss'], label='loss')
  plt.plot(history.history['val_loss'], label='val_loss')
  plt.ylim([0, 10])
  plt.xlabel('Epoch')
  plt.ylabel('Error [Distance]')
  plt.legend()
  plt.grid(True)

plot_loss(history)

pd.DataFrame(history.history).plot(figsize=(8,5))
plt.show()

test_results = {}

test_results['location_model'] = model.evaluate(
    X_test,
    y_test, verbose=0)

print(test_results)

In [ ]:
test_predictions = model.predict(X_test).flatten()

a = plt.axes(aspect='equal')
plt.scatter(y_test, test_predictions)
plt.xlabel('True Values [Distance]')
plt.ylabel('Predictions [Distance]')
lims = [0, 21]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims, lims)
plt.show()

error = test_predictions - np.array(y_test)
plt.hist(error, bins=25)
plt.xlabel('Prediction Error [Distance]')
_ = plt.ylabel('Count')
plt.show()

mae = mean_absolute_error(y_test, test_predictions)
mse = mean_squared_error(y_test, test_predictions)
rmse = np.sqrt(mean_squared_error(y_test, test_predictions))

# performance metrics on the test data
print('Mean Absolute Error:', mae)
print('Mean Squared Error:', mse)
print('Root Mean Squared Error:', rmse)
print()

model.summary()

<h1> Tensor flow convert to ONNX model for AIDGE convertion </h1>

In [ ]:
model.save(('Trilateration_'+str(title)+'.keras'))

onnx_model = onnxmltools.convert_keras(model) 

onnxmltools.utils.save_model(onnx_model, ('C:/Users/chiel/Desktop/Thesis project/Code/FederatedSpatialModelling/ML models/Trilateration/Trilateration_'+str(title)+'.onnx'))